# CPSC 368 SQL Sample

https://www.students.cs.ubc.ca/~cs-368/resources/connecting-to-db.html

To upload tables: 

- Import "knm_datasetup.sql" to SSH server: `scp "<PATH_TO_START>/knm_datasetup.sql" <CWL>@remote.students.cs.ubc.ca:/<PATH_TO_END>/`
- SSH connect to UBC department servers: `ssh <CWL>@remote.students.cs.ubc.ca`
- Move to location of "knm_datasetup.sql": `cd <PATH_TO_SQL>`
- Enter MySQL within SSH: `rlwrap sqlplus ora_<CWL>@stu`
    - Password: `<studentnum>`
- In MySQL, run `start knm_datasetup.sql;` to run all SQL imports

To run SQL in notebook (only tested within Jupyter Lab):

- Open terminal
- Run `ssh -l <CWL> -L 127.0.0.1:1522:dbhost.students.cs.ubc.ca:1522 remote.students.cs.ubc.ca` to access SSH server

In [1]:
import oracledb
import pandas as pd

In [2]:
CWL = ...
studentnum = ...

dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    # SQL code
    for row in cur.execute(
        """
        SELECT * FROM KFF2019_adult
        """
        ):
        print(row)
except Exception as e:
    print(f"Error executing SQL query: {e}")
finally: 
    cur.close()
    connection.close()

DatabaseError: ORA-01017: invalid username/password; logon denied
Help: https://docs.oracle.com/error-help/db/ora-01017/

In [ ]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    cur.execute("SELECT COUNT(*) FROM ALL_VIEWS WHERE VIEW_NAME = 'KFF2019_NEW'")
    view_exists = cur.fetchone()[0]

    if view_exists:
        cur.execute("DROP VIEW KFF2019_NEW CASCADE CONSTRAINTS")
    
    # Execute SQL query
    cur.execute(
        """
        CREATE VIEW KFF2019_NEW AS 
        SELECT kffa1."Location" AS Location, kffa1."Uninsured" AS All_Uninsured, kfff1."Uninsured" AS Female_Uninsured, kffm1."Uninsured" AS Male_Uninsured
        FROM KFF2019_adult kffa1
        INNER JOIN KFF2019_female kfff1 ON kffa1."Location" = kfff1."Location"
        INNER JOIN KFF2019_male kffm1 ON kffa1."Location" = kffm1."Location"
        WHERE kffa1."Location" != 'United States'
        """
    )

    cur.execute(
        """
        SELECT * 
        FROM KFF2019_NEW
        """
    )
    
    # Fetch results (only if SELECT is used)
    rows = cur.fetchall()
    
    # Get column names from cursor
    columns = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    df = pd.DataFrame(rows, columns=columns)
    
    print("Query executed successfully!")
    
except Exception as e:
    print(f"Error executing SQL query: {e}")

finally:
    # Close cursor
    cur.close()
    connection.close()

In [ ]:
display(df.head())

In [ ]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    cur.execute("SELECT COUNT(*) FROM ALL_VIEWS WHERE VIEW_NAME = 'USCDI'")
    view_exists = cur.fetchone()[0]

    if view_exists:
        cur.execute("DROP VIEW USCDI CASCADE CONSTRAINTS")
    
    # Execute SQL query
    cur.execute(
        """
        CREATE VIEW USCDI AS 
        SELECT cdif1."YearStart", cdif1."YearEnd", cdif1."LocationDesc", cdif1."Topic", cdif1."Question", cdif1."DataValueUnit", cdif1."DataValueType", cdif1."DataValue", cdif1."StratificationCategory1", cdif1."Stratification1"
        FROM USCDI_FILTER cdif1
        """
    )

    cur.execute(
        """
        SELECT * 
        FROM USCDI_FILTER
        """
    )
    
    # Fetch results (only if SELECT is used)
    rows2 = cur.fetchall()
    
    # Get column names from cursor
    columns2 = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    df2 = pd.DataFrame(rows2, columns=columns2)
    
    print("Query executed successfully!")
    
except Exception as e:
    print(f"Error executing SQL query: {e}")

finally:
    # Close cursor and connection
    cur.close()
    connection.close()

In [ ]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    cur.execute("SELECT COUNT(*) FROM ALL_VIEWS WHERE VIEW_NAME = 'USCDI'")
    view_exists = cur.fetchone()[0]

    if view_exists:
        cur.execute("DROP VIEW USCDI CASCADE CONSTRAINTS")
    
    # Execute SQL query
    cur.execute(
        """
        CREATE VIEW USCDI AS 
        SELECT USCDI_MID."YearStart", USCDI_MID."YearEnd", USCDI_MID."LocationDesc", USCDI_MID."Topic", USCDI_MID."Question", 
               USCDI_MID."DataValueUnit", USCDI_MID."DataValueType", USCDI_MID."DataValue", USCDI_MID."StratificationCategory1", USCDI_MID."Stratification1", 
               USCDI_MID."Has2019", 
               USCDI_MID."Range", 
               (USCDI_MID."DataValue" / USCDI_MID."Range") AS "AvgDataValue"
        FROM (
            SELECT cdif1."YearStart", cdif1."YearEnd", cdif1."LocationDesc", cdif1."Topic", cdif1."Question", 
                   cdif1."DataValueUnit", cdif1."DataValueType", cdif1."DataValue", 
                   cdif1."StratificationCategory1", cdif1."Stratification1", 
                   CAST(
                       CASE 
                           WHEN ((cdif1."YearStart" <= 2019) AND (cdif1."YearEnd" >= 2019)) THEN 1
                           ELSE 0
                       END AS NUMBER(1, 0)
                   ) AS "Has2019", 
                   CAST(
                       (cdif1."YearEnd" - cdif1."YearStart" + 1) AS NUMBER(2, 0)
                   ) AS "Range"
            FROM USCDI_filter cdif1
            WHERE cdif1."LocationDesc" != 'United States'
        ) USCDI_MID
        """
    )

    cur.execute(
        """
        SELECT * 
        FROM USCDI
        """
    )
    
    # Fetch results (only if SELECT is used)
    rows2 = cur.fetchall()
    
    # Get column names from cursor
    columns2 = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    df2 = pd.DataFrame(rows2, columns=columns2)
    
    print("Query executed successfully!")
    
except Exception as e:
    print(f"Error executing SQL query: {e}")

finally:
    # Close cursor and connection
    cur.close()
    connection.close()

In [ ]:
display(df2.head())